In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os

In [2]:
train_label = []
train_data = []
test_label = []
test_data = []

In [3]:
label_list = ["angry","disgust","fear", "happy", "sad", "surprise", "neutral"]
for i in label_list:
    for j in os.listdir(f'train/{i}/'):
        train_label.append(label_list.index(i))
        train_data.append(f'train/{i}/{j}')

In [4]:
label_list = ["angry","disgust","fear", "happy", "sad", "surprise", "neutral"]
for i in label_list:
    for j in os.listdir(f'test/{i}/'):
        test_label.append(label_list.index(i))
        test_data.append(f'test/{i}/{j}')

In [5]:
df_train = pd.DataFrame(data=train_data, columns=['path'])
df_train['label'] = train_label

In [6]:
df_test = pd.DataFrame(data=test_data, columns=['path'])
df_test['label'] = test_label

In [24]:
from keras import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout

In [11]:
X = df_train.drop('label', axis=1)
y = df_train['label']

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [14]:
X_train

,path
22453,train/surprise/Training_63790868.jpg
13212,train/happy/Training_68630514.jpg
22655,train/surprise/Training_68787983.jpg
21347,train/surprise/Training_31741324.jpg
3589,train/angry/Training_91498767.jpg
...,...
21575,train/surprise/Training_38504835.jpg
5390,train/fear/Training_31740082.jpg
860,train/angry/Training_29393434.jpg
15795,train/sad/Training_11007176.jpg


In [15]:
X_train_data = []
for i in range(len(X_train)):
    try:
        image = cv2.imread(X_train.values[i][0])
        image = image / 255
        X_train_data.append(image)
    except:
        y_train = y_train.drop(i, axis=0)
        continue
X_train_data = np.array(X_train_data)

In [16]:
X_test = df_test.drop('label', axis=1)
y_test = df_test['label']

In [17]:
X_test_data = []
for i in range(len(X_test)):
    try:
        image = cv2.imread(X_test.values[i][0])
        image = image / 255
        X_test_data.append(image)
    except:
        y_test = y_test.drop(i, axis=0)
        continue
X_test_data = np.array(X_test_data)

In [18]:
X_val_data = []
for i in range(len(X_val)):
    try:
        image = cv2.imread(X_val.values[i][0])
        image = image / 255
        X_val_data.append(image)
    except:
        y_val = y_val.drop(i, axis=0)
        continue
X_val_data = np.array(X_val_data)

In [19]:
from keras.utils import to_categorical

In [20]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [30]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', input_shape=(48, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), padding='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(50, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(X_train_data, y_train, batch_size=32, epochs=20, validation_data=(X_val_data, y_val))

Epoch 1/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.5929 - loss: 1.0934 - val_accuracy: 0.4203 - val_loss: 1.6557
Epoch 2/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6196 - loss: 1.0494 - val_accuracy: 0.4189 - val_loss: 1.6060
Epoch 3/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6308 - loss: 1.0049 - val_accuracy: 0.4063 - val_loss: 1.6590
Epoch 4/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6475 - loss: 0.9625 - val_accuracy: 0.4168 - val_loss: 1.7181
Epoch 5/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.6535 - loss: 0.9486 - val_accuracy: 0.4157 - val_loss: 1.7510
Epoch 6/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.6671 - loss: 0.9072 - val_accuracy: 0.4220 - val_loss: 1.7563
Epoch 7/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.6857 - loss: 0.8633 - val_accuracy: 0.4140 - val_loss: 1.6875
Epoch 8/20
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6959 - loss: 0.8392 - 

In [34]:
y_pred = model.predict(X_test_data)

225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [36]:
y_pred = [np.argmax(i) for i in y_pred]

In [38]:
y_pred = np.array(y_pred)

In [40]:
from sklearn.metrics import classification_report

In [41]:
y_test_ = [np.argmax(i) for i in y_test]

In [42]:
y_test_ = np.array(y_test_)

In [43]:
print(classification_report(y_pred, y_test_))

              precision    recall  f1-score   support

           0       0.26      0.32      0.29       768
           1       0.03      0.75      0.05         4
           2       0.31      0.23      0.26      1362
           3       0.55      0.57      0.56      1725
           4       0.33      0.30      0.31      1381
           5       0.49      0.56      0.52       722
           6       0.36      0.36      0.36      1216

    accuracy                           0.39      7178
   macro avg       0.33      0.44      0.34      7178
weighted avg       0.39      0.39      0.39      7178



In [72]:
import pickle

In [73]:
with open('mymodel.pkl', 'wb') as file:
    pickle.dump(model, file)